<a href="https://colab.research.google.com/github/deniztahmouresi/interviewTask/blob/develop/Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import numpy as np
drive.mount('/content/drive')
dataset = np.load('/content/drive/MyDrive/Colab Notebooks/interviewTask/data.npz')


Mounted at /content/drive


In [17]:
x_train = dataset['X_train']
y_train = dataset['y_train']
x_test = dataset['X_test']
y_test = dataset['y_test']

In [18]:
# Data normalize to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [19]:
count = np.zeros((10,), dtype=int)
for val in y_train:
    val = int(val)
    count[val]+=1
print(count*100/sum(count))
print(count)

[ 8.81923764 20.07742704  8.871352    9.1289458   4.34931507  8.07176891
  8.81179273 18.65693865  4.35527099  8.85795116]
[ 5923 13484  5958  6131  2921  5421  5918 12530  2925  5949]


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define data preparation
datagen = ImageDataGenerator(rotation_range = 20)
# configure batch size and retrieve one batch of images from specific classes
classes = [0, 2, 3, 4, 5, 6, 8, 9]
i=0
x_train_new = np.ndarray(shape=(40296,28, 28, 1), dtype=float)
y_train_new = np.ndarray(shape=(40296,), dtype=int)
for X_batch, y_batch in datagen.flow(x_train, y_train, batch_size=1, shuffle=True):
    if(y_batch[0] in classes):
      x_train_new[i] = X_batch
      y_train_new[i] = y_batch
      i += 1
      if i > 40295:
        break
    else:
      continue
print(x_train_new.max())
print(x_train_new.min())
print(x_train_new.shape)
print(y_train_new.shape)

1.0
0.0
(40296, 28, 28, 1)
(40296,)


In [21]:
# configure batch size and retrieve one batch of images from specific classes
classes = [4, 8]
i=0
x_train_new2 = np.ndarray(shape=(10000,28, 28, 1), dtype=float)
y_train_new2 = np.ndarray(shape=(10000,), dtype=int)
for X_batch, y_batch in datagen.flow(x_train, y_train, batch_size=1, shuffle=True):
    if(y_batch[0] in classes):
      x_train_new2[i] = X_batch
      y_train_new2[i] = y_batch
      i += 1
      if i > 9999:
        break
    else:
      continue
print(x_train_new2.max())
print(x_train_new2.min())
print(x_train_new2.shape)
print(y_train_new2.shape)

1.0
0.0
(10000, 28, 28, 1)
(10000,)


In [22]:
x_train = np.concatenate((x_train, x_train_new), axis=0)
x_train = np.concatenate((x_train, x_train_new2), axis=0)
y_train = np.concatenate((y_train, y_train_new), axis=0)
y_train = np.concatenate((y_train, y_train_new2), axis=0)

print(x_train.shape)
print(y_train.shape)

(117456, 28, 28, 1)
(117456,)


In [23]:
count = np.zeros((10,), dtype=int)
for val in y_train:
    val = int(val)
    count[val]+=1
print(count*100/sum(count))
print(count)

[ 9.9833129  11.48004359 10.04631522 10.33493393  9.14980929  9.1395927
  9.95606866 10.66782455  9.22473096 10.01736821]
[11726 13484 11800 12139 10747 10735 11694 12530 10835 11766]


In [24]:
from tensorflow import keras
from tensorflow.keras import layers

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [25]:
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", kernel_initializer='he_uniform'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", kernel_initializer='he_uniform'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(100, activation="relu", kernel_initializer='he_uniform'),
        layers.Dense(10, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 100)              

In [26]:
from tensorflow.keras.optimizers import SGD

batch_size = 128
epochs = 15
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["categorical_accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
826/826 [==============================] - 52s 62ms/step - loss: 0.1168 - categorical_accuracy: 0.9645 - val_loss: 0.0600 - val_categorical_accuracy: 0.9806
Epoch 2/15
826/826 [==============================] - 50s 60ms/step - loss: 0.0294 - categorical_accuracy: 0.9910 - val_loss: 0.0646 - val_categorical_accuracy: 0.9797
Epoch 3/15
826/826 [==============================] - 50s 60ms/step - loss: 0.0199 - categorical_accuracy: 0.9940 - val_loss: 0.0384 - val_categorical_accuracy: 0.9869
Epoch 4/15
826/826 [==============================] - 52s 63ms/step - loss: 0.0149 - categorical_accuracy: 0.9956 - val_loss: 0.0531 - val_categorical_accuracy: 0.9812
Epoch 5/15
826/826 [==============================] - 50s 61ms/step - loss: 0.0117 - categorical_accuracy: 0.9964 - val_loss: 0.0287 - val_categorical_accuracy: 0.9896
Epoch 6/15
826/826 [==============================] - 50s 60ms/step - loss: 0.0098 - categorical_accuracy: 0.9970 - val_loss: 0.0240 - val_categorical_accuracy:

In [27]:
from sklearn.metrics import classification_report

#Define metrics report function
def evaluate_model(x_test, y_test, title):
  y_pred = model.predict(x_test, verbose=0)
  score = model.evaluate(x_test, y_test, verbose=0)
  print("Report metrics for CNN " + title)
  print("Test loss:", score[0])
  print()
  print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
evaluate_model(x_test, y_test, "with rotation augmentation")

Report metrics for CNN with rotation augmentation
Test loss: 0.723524272441864

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       980
           1       0.89      1.00      0.94      1135
           2       0.99      0.57      0.73      1032
           3       0.73      1.00      0.85      1010
           4       0.75      0.99      0.86       982
           5       1.00      0.53      0.69       892
           6       1.00      0.88      0.94       958
           7       0.83      1.00      0.90      1028
           8       0.68      0.99      0.81       974
           9       1.00      0.43      0.60      1009

    accuracy                           0.84     10000
   macro avg       0.88      0.84      0.83     10000
weighted avg       0.88      0.84      0.83     10000



In [36]:
from scipy import signal

x_train = dataset['X_train']
y_train = dataset['y_train']
x_test = dataset['X_test']
y_test = dataset['y_test']

X_train = np.ndarray(shape=(len(x_train),28, 28) , dtype=int)
i = 0
for img in x_train:
  X_train[i] = signal.medfilt2d(img)
  i += 1
X_test = np.ndarray(shape=(len(x_test),28, 28) , dtype=int)
i = 0
for img in x_test:
  X_test[i] = signal.medfilt2d(img)
  i += 1

In [37]:
# Data normalize to the [0, 1] range
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [38]:
# configure batch size and retrieve one batch of images from specific classes
classes = [0, 2, 3, 4, 5, 6, 8, 9]
i=0
x_train_new = np.ndarray(shape=(40296,28, 28, 1), dtype=float)
y_train_new = np.ndarray(shape=(40296,), dtype=int)
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=1, shuffle=True):
    if(y_batch[0] in classes):
      x_train_new[i] = X_batch
      y_train_new[i] = y_batch
      i += 1
      if i > 40295:
        break
    else:
      continue
print(x_train_new.max())
print(x_train_new.min())
print(x_train_new.shape)
print(y_train_new.shape)

# configure batch size and retrieve one batch of images from specific classes
classes = [4, 8]
i=0
x_train_new2 = np.ndarray(shape=(10000,28, 28, 1), dtype=float)
y_train_new2 = np.ndarray(shape=(10000,), dtype=int)
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=1, shuffle=True):
    if(y_batch[0] in classes):
      x_train_new2[i] = X_batch
      y_train_new2[i] = y_batch
      i += 1
      if i > 9999:
        break
    else:
      continue
print(x_train_new2.max())
print(x_train_new2.min())
print(x_train_new2.shape)
print(y_train_new2.shape)

X_train = np.concatenate((X_train, x_train_new), axis=0)
X_train = np.concatenate((X_train, x_train_new2), axis=0)
y_train = np.concatenate((y_train, y_train_new), axis=0)
y_train = np.concatenate((y_train, y_train_new2), axis=0)

print(X_train.shape)
print(y_train.shape)

1.0
0.0
(40296, 28, 28, 1)
(40296,)
1.0
0.0
(10000, 28, 28, 1)
(10000,)
(117456, 28, 28, 1)
(117456,)


In [39]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
826/826 [==============================] - 48s 58ms/step - loss: 0.0433 - categorical_accuracy: 0.9855 - val_loss: 0.1099 - val_categorical_accuracy: 0.9609
Epoch 2/15
826/826 [==============================] - 48s 58ms/step - loss: 0.0298 - categorical_accuracy: 0.9904 - val_loss: 0.0862 - val_categorical_accuracy: 0.9691
Epoch 3/15
826/826 [==============================] - 46s 55ms/step - loss: 0.0230 - categorical_accuracy: 0.9925 - val_loss: 0.0296 - val_categorical_accuracy: 0.9905
Epoch 4/15
826/826 [==============================] - 47s 57ms/step - loss: 0.0192 - categorical_accuracy: 0.9937 - val_loss: 0.0214 - val_categorical_accuracy: 0.9934
Epoch 5/15
826/826 [==============================] - 46s 55ms/step - loss: 0.0157 - categorical_accuracy: 0.9950 - val_loss: 0.0539 - val_categorical_accuracy: 0.9811
Epoch 6/15
826/826 [==============================] - 47s 57ms/step - loss: 0.0135 - categorical_accuracy: 0.9955 - val_loss: 0.0352 - val_categorical_accuracy:

In [40]:
evaluate_model(x_test, y_test, "with rotation augmentation and noise filtering")

Report metrics for CNN with rotation augmentation and noise filtering
Test loss: 43.556400299072266

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      1.00      0.99      1032
           3       0.85      1.00      0.92      1010
           4       0.97      0.99      0.98       982
           5       0.99      0.73      0.84       892
           6       1.00      0.92      0.96       958
           7       0.99      0.99      0.99      1028
           8       0.94      0.98      0.96       974
           9       0.95      0.99      0.97      1009

    accuracy                           0.96     10000
   macro avg       0.97      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

